In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import io
import imageio
from IPython.display import Image, display
from ipywidgets import widgets, Layout, HBox
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from matplotlib.figure import Figure
import math
import imageio
import datetime
from os import walk
from scipy import stats

from ConvLSTM import ConvLSTM
from SlimAutoencoder import SlimAutoencoderBuilder
from rea_cd_autoencoder import REA_CD_Autoencoder
from dataloader import hillsborough_county_rea_cd_encoded as hc_rea_cd_encoded
from datawrangler import DataWrangler


# random forest
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, plot_tree
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

from sklearn.metrics import mean_squared_error

import multiprocessing
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
residential = pd.read_pickle('../Datasets/residential_conveyances_jan2000_and_up.pickle')
residential.shape

(1286882, 64)

In [4]:
dw = DataWrangler()
df = dw.prepare(residential)
display(residential.head())
display(df.head())

,PIN_x,FOLIO,DOR_CODE,NBHC_x,S_DATE_x,VI_x,QU,REA_CD,S_AMT_x,SUB_x,STR,S_TYPE,OR_BK,OR_PG,GRANTOR,GRANTEE,DOC_NUM,TYPE,PIN_y,DOR_C,OWNER,ADDR_1,ADDR_2,CITY,STATE,ZIP,COUNTRY,SUB_y,SITE_ADDR,SITE_CITY,SITE_ZIP,LEGAL1,LEGAL2,LEGAL3,LEGAL4,DBA,STRAP,tBEDS,tBATHS,tSTORIES,tUNITS,tBLDGS,TAXDIST,JUST,LAND,BLDG,EXF,ACT,EFF,HEAT_AR,ASD_VAL,TAX_VAL,MUNI,SD1,SD2,TIF,BASE,S_DATE_y,VI_y,S_AMT_y,ACREAGE,NBHC_y,Edit_dt,geometry
518496,A-28-27-19-1CE-000006-00005.0,0339824370,0100,216003.0,2000-01-01,I,U,None,110800.0,1CE,282719,WD,10533,0877,NELSON JOANNE B,NELSON JOANNE B,2000372038,None,A-28-27-19-1CE-000006-00005.0,0100,JOANNE B NELSON TRUSTEE / EL AL,4904 LONDONDERRY DR,None,TAMPA,FL,33647-1334,None,1CE,4904 LONDONDERRY DR,TAMPA,33647,TAMPA PALMS AREA 2 UNIT 7D,LOT 5 BLOCK 6,None,None,None,1927281CE000006000050A,4.0,3.0,1.0,1.0,1.0,TA,342327.0,96705.0,216374.0,29248.0,1991.0,2005.0,2434.0,283132.0,233132.0,A,012,None,None,1999.0,1998-07-01,I,240000.0,0.366779,216003.0,2019-03-01,"POLYGON ((525221.538 1371469.238, 525142.066 1..."
204670,U-02-29-17-0FH-000002-00023.1,0114050000,0100,208001.0,2000-01-01,I,Q,None,63500.0,0FH,022917,WD,10455,0531,LESTER RAYMOND G,OLIVERA ALEXIS,2000320171,None,U-02-29-17-0FH-000002-00023.1,0100,EDMONTON CHINA HOME LTD,4790 MEADFEILD RD,None,WEST VANCOUVER BC V7W 2Y3,None,None,CANADA,0FH,4718 EL DORADO DR,TAMPA,33615,TOWN'N COUNTRY PARK UNIT NO 01,S 1 FT OF LOT 23 AND LOT 24 BLOCK 2,None,None,None,1729020FH000002000231U,4.0,2.0,1.0,1.0,1.0,U,150285.0,39769.0,109127.0,1389.0,1959.0,1989.0,1331.0,149259.0,149259.0,U,None,None,None,0.0,2017-10-30,I,175000.0,0.144635,208001.0,2020-09-22,"POLYGON ((473022.713 1328707.108, 473022.162 1..."
1674793,A-34-29-18-5PI-000000-00004.0,1272645508,0106,803362.0,2000-01-01,V,U,05,151000.0,5PI,342918,WD,10005,1174,None,LAND IN SOUTH TAMPA,2000008849,None,A-34-29-18-5PI-000000-00004.0,0106,SHAWN P MC CONNON,3014 W GROVEWOOD CT APT H,None,TAMPA,FL,33629-8894,None,5PI,3014 W GROVEWOOD CT H,TAMPA,33629,HACIENDA ANNA MARIA TOWNHOMES,LOT 4,None,None,None,1829345PI000000000040A,2.0,2.5,3.0,1.0,0.0,TA,295087.0,29472.0,265251.0,364.0,2001.0,2011.0,940.0,132025.0,82025.0,A,None,None,None,2013.0,2012-09-26,I,135000.0,0.028191,203002.0,2019-07-31,"POLYGON ((496888.344 1302311.612, 496858.803 1..."
1802813,A-19-28-19-45M-000015-00001.0,1463400000,0100,214003.0,2000-01-01,V,U,05,1744.0,45M,192819,TD,10446,1893,REALTY ASSET PROPERTIES INC,GERLACH PHILIP E JR TRUSTEE,2000315424,None,A-19-28-19-45M-000015-00001.0,0100,HC OF TAMPA BAY LLC,12001 N NEBRASKA AVE,None,TAMPA,FL,33612-5343,None,45M,8415 N 10TH ST,TAMPA,33604,SULPHUR SPRINGS ADDITION,LOT 1 AND W 1/2 CLOSED ALLEY ABUTTING,THEREON BLOCK 15,None,None,19281945M000015000010A,3.0,2.0,1.0,1.0,1.0,TA,125679.0,22725.0,102954.0,0.0,2002.0,2011.0,1200.0,112467.0,112467.0,A,None,None,None,0.0,2018-11-16,I,99000.0,0.112013,214003.0,2019-03-23,"POLYGON ((511234.743 1342556.267, 511137.063 1..."
1674789,A-34-29-18-5PI-000000-00003.0,1272645506,0106,803362.0,2000-01-01,V,U,05,151000.0,5PI,342918,WD,10005,1174,None,LAND IN SOUTH TAMPA,2000008849,None,A-34-29-18-5PI-000000-00003.0,0106,RICHARD K MCHUGH JR,3014 W GROVEWOOD CT APT G,None,TAMPA,FL,33629-8894,None,5PI,3014 W GROVEWOOD CT G,TAMPA,33629,HACIENDA ANNA MARIA TOWNHOMES,LOT 3,None,None,None,1829345PI000000000030A,2.0,2.5,3.0,1.0,0.0,TA,273838.0,27384.0,246454.0,0.0,2001.0,2011.0,940.0,229640.0,229640.0,A,None,None,None,0.0,2001-05-10,I,160000.0,0.014286,203002.0,2019-07-31,"POLYGON ((496858.934 1302355.962, 496859.005 1..."


,FOLIO,S_DATE_x,REA_CD,S_AMT_x,STR,tBEDS,tBATHS,tSTORIES,tUNITS,tBLDGS,JUST,LAND,BLDG,EXF,ACT,EFF,HEAT_AR,ASD_VAL,TAX_VAL,BASE,ACREAGE,NBHC_x_201001.0,NBHC_x_201002.0,NBHC_x_201003.0,NBHC_x_201004.0,NBHC_x_201005.0,NBHC_x_201006.0,NBHC_x_201007.0,NBHC_x_202001.0,NBHC_x_202002.0,NBHC_x_202003.0,NBHC_x_202004.0,NBHC_x_202005.0,NBHC_x_202006.0,NBHC_x_202007.0,NBHC_x_202008.0,NBHC_x_202009.0,NBHC_x_203001.0,NBHC_x_203002.0,NBHC_x_203003.0,NBHC_x_203004.0,NBHC_x_203005.0,NBHC_x_203006.0,NBHC_x_203007.0,NBHC_x_203008.0,NBHC_x_203009.0,NBHC_x_203010.0,NBHC_x_203011.0,NBHC_x_203012.0,NBHC_x_203013.0,NBHC_x_203014.0,NBHC_x_203015.0,NBHC_x_204001.0,NBHC_x_204002.0,NBHC_x_204003.0,NBHC_x_204004.0,NBHC_x_204005.0,NBHC_x_204006.0,NBHC_x_204007.0,NBHC_x_204008.0,NBHC_x_205001.0,NBHC_x_205002.0,NBHC_x_205003.0,NBHC_x_205004.0,NBHC_x_205005.0,NBHC_x_205006.0,NBHC_x_205007.0,NBHC_x_205008.0,NBHC_x_205009.0,NBHC_x_205010.0,NBHC_x_205011.0,NBHC_x_205012.0,NBHC_x_205013.0,NBHC_x_205014.0,NBHC_x_206001.0,NBHC_x_206002.0,NBHC_x_206003.0,NBHC_x_206004.0,NBHC_x_206005.0,NBHC_x_206006.0,NBHC_x_206007.0,NBHC_x_206008.0,NBHC_x_206009.0,NBHC_x_206010.0,NBHC_x_206011.0,NBHC_x_206012.0,NBHC_x_206013.0,NBHC_x_206014.0,NBHC_x_206015.0,NBHC_x_206016.0,NBHC_x_206017.0,NBHC_x_206018.0,NBHC_x_206019.0,NBHC_x_206020.0,NBHC_x_207001.0,NBHC_x_207002.0,NBHC_x_207003.0,NBHC_x_207004.0,NBHC_x_207005.0,NBHC_x_207006.0,NBHC_x_208001.0,NBHC_x_208002.0,NBHC_x_208003.0,NBHC_x_208004.0,NBHC_x_208005.0,NBHC_x_208006.0,NBHC_x_208007.0,NBHC_x_208008.0,NBHC_x_208009.0,NBHC_x_208010.0,NBHC_x_209001.0,NBHC_x_209002.0,NBHC_x_209003.0,NBHC_x_209004.0,NBHC_x_209005.0,NBHC_x_209006.0,NBHC_x_209007.0,NBHC_x_209008.0,NBHC_x_209009.0,NBHC_x_209010.0,NBHC_x_209012.0,NBHC_x_209013.0,NBHC_x_209014.0,NBHC_x_209015.0,NBHC_x_209016.0,NBHC_x_210001.0,NBHC_x_210002.0,NBHC_x_210003.0,NBHC_x_210004.0,NBHC_x_210005.0,NBHC_x_210006.0,NBHC_x_210007.0,NBHC_x_210008.0,NBHC_x_210009.0,NBHC_x_210010.0,NBHC_x_210011.0,NBHC_x_210012.0,NBHC_x_210013.0,NBHC_x_210014.0,NBHC_x_210015.0,NBHC_x_210016.0,NBHC_x_211001.0,NBHC_x_211002.0,NBHC_x_211003.0,NBHC_x_211004.0,NBHC_x_211005.0,NBHC_x_211006.0,NBHC_x_211007.0,NBHC_x_211008.0,NBHC_x_211009.0,NBHC_x_211010.0,NBHC_x_211011.0,NBHC_x_212001.0,NBHC_x_212002.0,NBHC_x_212003.0,NBHC_x_212004.0,NBHC_x_212005.0,NBHC_x_212006.0,NBHC_x_212007.0,NBHC_x_212008.0,NBHC_x_212009.0,NBHC_x_212010.0,NBHC_x_213001.0,NBHC_x_213002.0,NBHC_x_213003.0,NBHC_x_213004.0,NBHC_x_213005.0,NBHC_x_213006.0,NBHC_x_213007.0,NBHC_x_214001.0,NBHC_x_214002.0,NBHC_x_214003.0,NBHC_x_214004.0,NBHC_x_214005.0,NBHC_x_214006.0,NBHC_x_214007.0,NBHC_x_214008.0,NBHC_x_214009.0,NBHC_x_215001.0,NBHC_x_215002.0,NBHC_x_215003.0,NBHC_x_215004.0,NBHC_x_215005.0,NBHC_x_215006.0,NBHC_x_215007.0,NBHC_x_215008.0,NBHC_x_215009.0,NBHC_x_215010.0,NBHC_x_215011.0,NBHC_x_215012.0,NBHC_x_215013.0,NBHC_x_215014.0,NBHC_x_215016.0,NBHC_x_215017.0,NBHC_x_215018.0,NBHC_x_215019.0,NBHC_x_215020.0,NBHC_x_215021.0,NBHC_x_215022.0,NBHC_x_215023.0,NBHC_x_216001.0,NBHC_x_216002.0,NBHC_x_216003.0,NBHC_x_216004.0,NBHC_x_216005.0,NBHC_x_216006.0,NBHC_x_216007.0,NBHC_x_216008.0,NBHC_x_216009.0,NBHC_x_216010.0,NBHC_x_216011.0,NBHC_x_216012.0,NBHC_x_217001.0,NBHC_x_217002.0,NBHC_x_217003.0,NBHC_x_217004.0,NBHC_x_217005.0,NBHC_x_217006.0,NBHC_x_217007.0,NBHC_x_218001.0,NBHC_x_218002.0,NBHC_x_218003.0,NBHC_x_218004.0,NBHC_x_218005.0,NBHC_x_218006.0,NBHC_x_218007.0,NBHC_x_218008.0,NBHC_x_218009.0,NBHC_x_218010.0,NBHC_x_218011.0,NBHC_x_219001.0,NBHC_x_219002.0,NBHC_x_219003.0,NBHC_x_219004.0,NBHC_x_219005.0,NBHC_x_219006.0,NBHC_x_219007.0,NBHC_x_220001.0,NBHC_x_220002.0,NBHC_x_220003.0,NBHC_x_220004.0,NBHC_x_220005.0,NBHC_x_220006.0,NBHC_x_220007.0,NBHC_x_220008.0,NBHC_x_220009.0,NBHC_x_220010.0,NBHC_x_220011.0,NBHC_x_220012.0,NBHC_x_220013.0,NBHC_x_220014.0,NBHC_x_220015.0,NBHC_x_220016.0,NBHC_x_221001.0,NBHC_x_221002.0,NBHC_x_221003.0,NBHC_x_221004.0,NBHC_x_221005.0,NBHC_x_221006.0,NBHC_x_221007.0,NBHC_x_221008.0,NBHC_x_221009.0,NBHC_x_22

In [5]:
df.shape

(1286573, 753)

In [7]:
df.to_pickle('../Datasets/residential_conveyances_jan2000_and_up_ohe.pickle')